In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
df = pd.read_excel("Employee Sample Data - A.xlsx")

In [3]:
print(df.head())

     EEID        Full Name                 Job Title Department  \
0  E02387      Emily Davis                Sr. Manger         IT   
1  E04105    Theodore Dinh       Technical Architect         IT   
2  E02572     Luna Sanders                  Director    Finance   
3  E02832  Penelope Jordan  Computer Systems Manager         IT   
4  E01639        Austin Vo               Sr. Analyst    Finance   

            Business Unit  Gender  Ethnicity   Age  Hire Date  Annual Salary  \
0  Research & Development  Female      Black  55.0 2016-04-08       141604.0   
1           Manufacturing    Male      Asian  59.0 1997-11-29        99975.0   
2     Speciality Products  Female  Caucasian  50.0 2006-10-26       163099.0   
3           Manufacturing  Female  Caucasian  26.0 2019-09-27        84913.0   
4           Manufacturing    Male      Asian  55.0 1995-11-20        95409.0   

   Bonus %        Country       City  Exit Date  
0     0.15  United States    Seattle 2021-10-16  
1     0.00      

In [4]:
print(df.columns)        

Index(['EEID', 'Full Name', 'Job Title', 'Department', 'Business Unit',
       'Gender', 'Ethnicity', 'Age', 'Hire Date', 'Annual Salary', 'Bonus %',
       'Country', 'City', 'Exit Date'],
      dtype='object')


In [5]:
print(df.info())         

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   EEID           1000 non-null   object        
 1   Full Name      998 non-null    object        
 2   Job Title      999 non-null    object        
 3   Department     998 non-null    object        
 4   Business Unit  1000 non-null   object        
 5   Gender         999 non-null    object        
 6   Ethnicity      993 non-null    object        
 7   Age            994 non-null    float64       
 8   Hire Date      993 non-null    datetime64[ns]
 9   Annual Salary  989 non-null    float64       
 10  Bonus %        992 non-null    float64       
 11  Country        998 non-null    object        
 12  City           998 non-null    object        
 13  Exit Date      85 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(9)
memory usage: 109.5+ KB
None


In [6]:
print(df.isnull().sum()) 

EEID               0
Full Name          2
Job Title          1
Department         2
Business Unit      0
Gender             1
Ethnicity          7
Age                6
Hire Date          7
Annual Salary     11
Bonus %            8
Country            2
City               2
Exit Date        915
dtype: int64


In [7]:
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

In [8]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip()

In [9]:
df["age"] = pd.to_numeric(df["age"], errors="coerce")
df["annual_salary"] = pd.to_numeric(df["annual_salary"], errors="coerce")

In [11]:
df["age"] = df["age"].fillna(df["age"].median())
df["annual_salary"] = df["annual_salary"].fillna(df["annual_salary"].median())

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
print(df.isnull().sum()) 

eeid               0
full_name          2
job_title          1
department         2
business_unit      0
gender             1
ethnicity          7
age                0
hire_date          7
annual_salary      0
bonus_%            8
country            2
city               2
exit_date        915
dtype: int64


In [15]:
df.loc[:4, "full_name"] = ["Employee 1", "Employee 2", "Employee 3", "Employee 4", "Employee 5"]
df.loc[:4, "age"] = [20, 21, 23, 28, 22]
df.loc[:4, "annual_salary"] = [5000, 3000, 4000, 4500, 3500]


In [16]:
highestSalary = df.loc[df["annual_salary"].idxmax()]
print(highestSalary)

eeid                          E04354
full_name               Raelynn Rios
job_title             Vice President
department                     Sales
business_unit          Manufacturing
gender                        Female
ethnicity                     Latino
age                             43.0
hire_date        2016-08-21 00:00:00
annual_salary               258498.0
bonus_%                         0.35
country                United States
city                        Columbus
exit_date                        NaT
Name: 989, dtype: object


In [17]:
group1 = df.groupby("department").agg(
    average_age=("age", "mean"),
    average_salary=("annual_salary", "mean")
)
print(group1)

                 average_age  average_salary
department                                  
Accounting         43.656250   123146.947917
Engineering        45.670886   109053.101266
Finance            44.708333   120894.616667
Human Resources    44.443548   117907.088710
IT                 44.070833    96634.050000
Marketing          43.216667   129663.033333
Sales              43.614286   111120.635714


In [18]:
group2 = df.groupby(["department", "ethnicity"]).agg(
    maxAge=("age", "max"),
    minAge=("age", "min"),
    medianSalary=("annual_salary", "median")
)
print(group2)

                           maxAge  minAge  medianSalary
department      ethnicity                              
Accounting      Asian        64.0    25.0      114893.0
                Black        51.0    30.0       91853.0
                Caucasian    62.0    26.0      121159.5
                Latino       62.0    26.0       92317.0
Engineering     Asian        65.0    25.0       91230.0
                Black        56.0    27.0       74412.0
                Caucasian    64.0    25.0       96057.0
                Latino       64.0    27.0       96818.0
Finance         Asian        64.0    22.0      124928.0
                Black        65.0    25.0      131652.0
                Caucasian    64.0    23.0       95045.0
                Latino       65.0    25.0      124553.0
Human Resources Asian        64.0    25.0      125871.5
                Black        55.0    25.0      142318.0
                Caucasian    64.0    26.0      100364.5
                Latino       65.0    29.0      1

In [19]:
with pd.ExcelWriter("EmployeeAnalysis.xlsx", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="cleanedData", index=False)
    group1.to_excel(writer, sheet_name="deptAverages")
    group2.to_excel(writer, sheet_name="deptEthnicityStats")